In [1]:
from autocode import Optimization

optimization: Optimization = Optimization(
    num_cpus=2,
    server_host="0.0.0.0",
    server_port=10000,
    dashboard_port=10001,
)

2024-08-06 17:30:44,299	INFO worker.py:1743 -- Started a local Ray instance. View the dashboard at http://172.25.4.192:8265 


Available resources:  {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'node:172.25.4.192': 1.0, 'CPU': 2.0, 'memory': 9103368192.0, 'object_store_memory': 4551684096.0, 'GPU': 1.0}
Server is healthy.


In [2]:
from autocode import OptimizationUseCase, ApplicationSetting
from autocode.container import ApplicationContainer
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "autocode"

application_container: ApplicationContainer = ApplicationContainer()
application_setting: ApplicationSetting = application_container.settings.application()
application_setting.num_cpus = 2
application_setting.server_host = "0.0.0.0"
application_setting.server_port = 10000
application_setting.dashboard_port = 10001
optimization_use_case: OptimizationUseCase = application_container.use_cases.optimization()
# optimization_use_case.reset()

In [ ]:
from python_on_whales import DockerClient

docker_client: DockerClient = optimization.deploy(
    compose_files=["./client-compose.yml"],
)

#0 building with "default" instance using docker driver

#1 [app-product internal] load build definition from .Dockerfile
#1 transferring dockerfile: 119B 0.0s done
#1 DONE 0.0s

#2 [app-account internal] load build definition from .Dockerfile
#2 transferring dockerfile: 119B 0.0s done
#2 DONE 0.0s

#3 [app-gateway internal] load build definition from .Dockerfile
#3 transferring dockerfile: 119B 0.0s done
#3 DONE 0.0s

#4 [app-gateway internal] load metadata for docker.io/library/golang:latest
#4 DONE 2.3s

#5 [app-account internal] load .dockerignore
#5 transferring context: 2B 0.0s done
#5 DONE 0.0s

#6 [app-gateway internal] load .dockerignore
#6 transferring context: 2B 0.0s done
#6 DONE 0.0s

#7 [app-product internal] load .dockerignore
#7 transferring context: 2B 0.0s done
#7 DONE 0.0s

#8 [app-product 1/5] FROM docker.io/library/golang:latest@sha256:86a3c48a61915a8c62c0e1d7594730399caa3feb73655dfe96c7bc17710e96cf
#8 DONE 0.0s

#9 [app-gateway internal] load build context
#9 tran

In [ ]:
containers = docker_client.ps(filters={"name": rf"^{docker_client.client_config.compose_project_name}_"})

In [ ]:
containers[0].network_settings.networks

In [30]:
from python_on_whales import Stack

docker_client.stack.remove(x=docker_client.client_config.compose_project_name)
docker_stack: Stack = docker_client.stack.deploy(
    name=docker_client.client_config.compose_project_name,
    compose_files=docker_client.client_config.compose_files,
)
for service in docker_stack.services():
    service.scale(
        new_scale=2,
    )

for container in docker_client.ps(filters={"name": rf"^{docker_client.client_config.compose_project_name}_"}):
    print(container.name)

DockerException: The command executed was `/usr/bin/docker stack remove 260cfbff09734102aaebd0a5631f1430`.
It returned with code 1
The content of stdout is 'Removing network 260cfbff09734102aaebd0a5631f1430_default
'
The content of stderr is 'Failed to remove network 9yc2pcidp21fxbfyggmhdysnj: Error response from daemon: network 9yc2pcidp21fxbfyggmhdysnj not foundFailed to remove some resources from stack: 260cfbff09734102aaebd0a5631f1430
'


In [7]:
from autocode import OptimizationObjective
from typing import Dict, Any, List
from autocode import OptimizationEvaluateRunResponse

objectives: List[OptimizationObjective] = [
    OptimizationObjective(
        type="minimize",
    ),

    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),

    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
    OptimizationObjective(
        type="minimize",
    ),
    OptimizationObjective(
        type="maximize",
    ),
]


def evaluator(inputs: List[OptimizationEvaluateRunResponse]) -> Dict[str, Any]:
    f_gateway: List[float] = []
    f_account: List[float] = []
    f_product: List[float] = []
    for input in inputs:
        if "gateway" in input.get_client().name:
            f_gateway = input.objectives
        elif "account" in input.get_client().name:
            f_account = input.objectives
        elif "product" in input.get_client().name:
            f_product = input.objectives

    output: Dict[str, Any] = {
        "F": f_gateway + f_account + f_product
    }

    return output


In [8]:
result = optimization.run(
    objectives=objectives,
    num_inequality_constraints=0,
    num_equality_constraints=0,
    evaluator=evaluator
)

OperationalError: (sqlite3.OperationalError) database is locked
[SQL: DELETE FROM cache WHERE (cache."key" LIKE ? || '%')]
[parameters: ('results',)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)